In [ ]:
#!/usr/bin/env python
"""
OpenPose BODY25 CSV → NTU-RGB+D PKL 변환 스크립트
경로 설정 및 일괄 처리
"""
from pathlib import Path
import os
import pickle
import numpy as np
import pandas as pd

# BODY_25 → COCO(17) 매핑
MAPPING_BODY25_TO_COCO17 = [
    0, 16, 15, 18, 17, 5, 2, 6, 3, 7, 4, 12, 9, 13, 10, 14, 11
]

def normalize_coordinates(keypoints, img_shape, method='0to1'):
    h, w = img_shape
    kp = keypoints.copy()
    if method == '0to1':
        kp[..., 0] /= w
        kp[..., 1] /= h
    elif method == 'center':
        kp[..., 0] = (kp[..., 0] - w/2) / (w/2)
        kp[..., 1] = (kp[..., 1] - h/2) / (h/2)
    elif method == 'skeleton_center':
        M, T, V, _ = kp.shape
        for m in range(M):
            for t in range(T):
                pts = kp[m, t, :, :2]
                mask = (pts != 0).all(axis=1)
                if mask.any():
                    valid = pts[mask]
                    cx, cy = valid.mean(axis=0)
                    bbox_w = valid[:,0].ptp()
                    bbox_h = valid[:,1].ptp()
                    scale = max(bbox_w, bbox_h)
                    if scale > 0:
                        kp[m, t, :, 0] = (kp[m, t, :, 0] - cx) / scale
                        kp[m, t, :, 1] = (kp[m, t, :, 1] - cy) / scale
    return kp

def filter_by_confidence(keypoints, scores, threshold=0.1):
    mask = scores < threshold
    keypoints[mask] = 0
    scores[mask] = 0
    return keypoints, scores

def interpolate_missing_keypoints(keypoints, scores):
    kp = keypoints.copy()
    M, T, V, C = kp.shape
    for m in range(M):
        for v in range(V):
            for c in range(C):
                series = kp[m, :, v, c]
                valid = series != 0
                if valid.any() and not valid.all():
                    idx = np.where(valid)[0]
                    interp = np.interp(np.arange(T), idx, series[idx])
                    kp[m, ~valid, v, c] = interp[~valid]
    return kp

def convert_csv_to_pkl(
    csv_path,
    pkl_path,
    frame_dir,
    label=0,
    img_shape=(1080, 1920),
    normalize_method='0to1',
    confidence_threshold=0.1,
    interpolate=False
):
    # Path 변환
    csv_path = Path(csv_path)
    pkl_path = Path(pkl_path)

    # 1. Load CSV
    df = pd.read_csv(csv_path)
    T = len(df)
    V25 = 25

    # 2. Parse to arrays
    kp25 = np.zeros((1, T, V25, 2), dtype=np.float32)
    score25 = np.zeros((1, T, V25), dtype=np.float32)
    for t, row in df.iterrows():
        for v in range(V25):
            x, y, s = row[3*v:3*v+3]
            kp25[0, t, v] = (x, y)
            score25[0, t, v] = s

    # 3. Confidence filtering
    if confidence_threshold > 0:
        kp25, score25 = filter_by_confidence(kp25, score25, confidence_threshold)

    # 4. Interpolation
    if interpolate:
        kp25 = interpolate_missing_keypoints(kp25, score25)

    # 5. Normalization
    kp25 = normalize_coordinates(kp25, img_shape, normalize_method)

    # 6. BODY25 → COCO17 conversion
    kp17 = kp25[:, :, MAPPING_BODY25_TO_COCO17, :]
    score17 = score25[:, :, MAPPING_BODY25_TO_COCO17]

    # 7. Package into NTU-style dict
    sample = {
        'frame_dir': frame_dir,
        'label': label,
        'img_shape': img_shape,
        'original_shape': img_shape,
        'total_frames': T,
        'keypoint': kp17,
        'keypoint_score': score17
    }
    data = {
        'split': {'xsub_val': [frame_dir]},
        'annotations': [sample]
    }

    # 8. Write to PKL
    with open(pkl_path, 'wb') as f:
        pickle.dump(data, f)

    print(f"[SUCCESS] {csv_path.name} → {pkl_path.name} ({T} frames)")
    return data

if __name__ == '__main__':
    # ✏️ 경로 설정
    BASE_DIR = Path(r"D:/golfDataset/스포츠 사람 동작 영상(골프)/Training/Public/male/train")
    CATEGORIES = ["balanced_true", "false"]

    for cat in CATEGORIES:
        csv_dir = BASE_DIR / cat / "crop_keypoint"
        pkl_dir = BASE_DIR / cat / "crop_pkl"
        pkl_dir.mkdir(parents=True, exist_ok=True)

        for csv_file in sorted(csv_dir.glob("*.csv")):
            frame_dir = csv_file.stem
            pkl_file = pkl_dir / f"{frame_dir}.pkl"
            print(f"[INFO] Converting {csv_file.name} → {pkl_file.name}")
            convert_csv_to_pkl(
                csv_path=csv_file,
                pkl_path=pkl_file,
                frame_dir=frame_dir,
                img_shape=(1080, 1920),
                normalize_method='0to1',
                confidence_threshold=0.1,
                interpolate=False
            )


[INFO] Converting 20201116_General_001_DOS_A_M40_MM_026.csv → 20201116_General_001_DOS_A_M40_MM_026.pkl


AttributeError: 'str' object has no attribute 'name'